In [1]:
# only when open in colab
!git init .
!git remote add origin https://github.com/coyote-bergstein/compression-using-neural-networks.git
!git pull origin speech-commands

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 41 (delta 14), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (41/41), 4.96 MiB | 3.51 MiB/s, done.
From https://github.com/coyote-bergstein/compression-using-neural-networks
 * branch            speech-commands -> FETCH_HEAD
 * [new branch]      speech-commands -> origin/speech-commands


In [2]:
!pip install pydub

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import librosa
import random

c:\Users\Dodek\anaconda3\envs\GlobalDsEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [69]:
# only when open in colab
import importlib
import source
from source.utils import generate_spectrogram, reconstruct_audio, normalize_standard
from source.constants import HOP_LENGTH, N_FFT, AUDIO_SAMPLE_RATE, WIN_LENGTH, WINDOW, CENTER, PAD_MODE, POWER, N_ITER, BATCH_SIZE, EPOCHS
importlib.reload(source.constants)
importlib.reload(source.utils)

<module 'source.utils' from 'c:\\Users\\Dodek\\Desktop\\AGH\\Kryptografia\\projekt\\compression-using-neural-networks\\source\\utils.py'>

## Dataset

In [4]:
train_ds = tfds.load('speech_commands', split='train')

In [11]:
for audio in train_ds.take(1):
    print(audio)

{'audio': <tf.Tensor: shape=(16000,), dtype=int16, numpy=array([-15, -18, -12, ...,  16,  21,  25], dtype=int16)>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=4>}


In [14]:
train_ds = train_ds.map(lambda x: x['audio'])

In [15]:
def get_min_max_length(dataset):
    min_length = float('inf')
    max_length = 0
    for audio in dataset:
        length = audio.shape[0]
        if length < min_length:
            min_length = length
        if length > max_length:
            max_length = length
    return min_length, max_length

In [16]:
min_length, max_length = get_min_max_length(train_ds)
print(f'Min length: {min_length}, max length: {max_length}')

Min length: 3413, max length: 16000


In [20]:
for audio in train_ds.take(1):
    print(audio.shape)

(16000,)


## Audio processing testing

In [30]:
import IPython.display as ipd
from source.utils import generate_spectrogram, reconstruct_audio
from source.constants import HOP_LENGTH, N_FFT, AUDIO_SAMPLE_RATE, TRACK_DURATION

for audio in train_ds.take(5):
  audio_normalized, mean, std = normalize_standard(audio.numpy())
  print("Audio shape:", audio_normalized.shape)
  S = generate_spectrogram(signal=audio_normalized, sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER)
  print("Spectrogram shape:", S.shape)
  reconstructed_signal = reconstruct_audio(S=S, sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, mean=mean, std=std, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER, n_iter=N_ITER)
  
  print("Original Audio:")
  ipd.display(ipd.Audio(audio_normalized, rate=AUDIO_SAMPLE_RATE))

  print("Reconstructed Audio shape:", reconstructed_signal.shape)
  print("Reconstructed Audio:")
  ipd.display(ipd.Audio(reconstructed_signal, rate=AUDIO_SAMPLE_RATE))

Audio shape: (16000,)
Spectrogram shape: (128, 32)
Reconstructed Audio shape: (15872,)
Original Audio:


Reconstructed Audio:


Audio shape: (16000,)
Spectrogram shape: (128, 32)
Reconstructed Audio shape: (15872,)
Original Audio:


Reconstructed Audio:


Audio shape: (16000,)
Spectrogram shape: (128, 32)
Reconstructed Audio shape: (15872,)
Original Audio:


...

Audio shape: (16000,)
Spectrogram shape: (128, 32)
Reconstructed Audio shape: (15872,)
Original Audio:


Reconstructed Audio:


Audio shape: (16000,)
Spectrogram shape: (128, 32)
Reconstructed Audio shape: (15872,)
Original Audio:


Reconstructed Audio:


2048 512


## Define model architecture
### TODO model architecture must be changed

In [49]:
n_mels = 128

In [38]:
def pad_audio(audio, length):
    audio_shape = tf.shape(audio)  # Get the dynamic shape of the audio tensor
    pad_length = length - audio_shape[0]  # Calculate the padding length
    padding = [[0, pad_length]]  # Create the padding specification as a list of lists
    padded_audio = tf.pad(audio, padding)  # Pad the audio tensor
    return padded_audio

padded_train_ds = train_ds.map(lambda x: pad_audio(audio=x, length=max_length))

min_length, max_length = get_min_max_length(padded_train_ds)
print(f'Min length: {min_length}, max length: {max_length}')
assert max_length == min_length

Min length: 16000, max length: 16000


In [39]:
for audio in padded_train_ds:
    assert audio.shape[0] == max_length

In [58]:
def preprocess(example):
    audio_normalized, mean, std = normalize_standard(example.numpy())
    S = generate_spectrogram(signal=audio_normalized, sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER)
    S = np.expand_dims(S, axis=-1)
    return S.astype(np.float32)

def preprocess_with_py_function(example):
    audio = tf.py_function(preprocess, [example], [tf.float32])
    # audio.set_shape([n_mels, None])
    return audio

In [59]:
ds_train = padded_train_ds.map(preprocess_with_py_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.map(lambda x: (x, x))
ds_train = ds_train.cache().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [62]:
for audio, label in ds_train:
    assert audio.shape[0] == BATCH_SIZE and audio.shape[3] == 1

KeyboardInterrupt: 

In [63]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [84]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_dense_autoencoder(input_shape, encoding_dim):
    input_layer = layers.Input(shape=input_shape)
    
    # Flatten the input
    x = layers.Flatten()(input_layer)
    
    # Encoder
    x = layers.Dense(encoding_dim, activation='relu')(x)
    encoded = layers.Dense(encoding_dim, activation='relu')(x)
    
    # Decoder
    x = layers.Dense(np.prod(input_shape), activation='relu')(encoded)
    decoded = layers.Reshape(input_shape)(x)
    
    autoencoder = models.Model(input_layer, decoded)
    return autoencoder


In [98]:
input_shape = (128, 32, 1)
encoding_dim = 1*1024  # Dimensionality of the encoding

autoencoder = create_dense_autoencoder(input_shape, encoding_dim)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 128, 32, 1)]      0         
                                                                 
 flatten_17 (Flatten)        (None, 4096)              0         
                                                                 
 dense_47 (Dense)            (None, 1024)              4195328   
                                                                 
 dense_48 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_49 (Dense)            (None, 4096)              4198400   
                                                                 
 reshape_17 (Reshape)        (None, 128, 32, 1)        0         
                                                                 
Total params: 9443328 (36.02 MB)
Trainable params: 9443328 

In [99]:
autoencoder.fit(ds_train, epochs=EPOCHS)

3329/3329 [==============================] - 1250s 375ms/step - loss: 2787082.2500


In [110]:
# Function to compress and decompress using the autoencoder
def compress_and_decompress(audio_normalized, mean, std):
    S = generate_spectrogram(signal=audio_normalized.numpy().astype(np.float32), sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER)
    S = np.expand_dims(S, axis=-1)
    S =  S.astype(np.float32)
    compressed_and_decompressed = autoencoder.predict(np.expand_dims(S, axis=0))
    
    compressed_and_decompressed = np.squeeze(compressed_and_decompressed, axis=-1)
    print("Compressed and decompressed shape:", compressed_and_decompressed.shape)
    
    # Reconstruct the audio signal from the spectrogram
    reconstructed_signal = reconstruct_audio(S=compressed_and_decompressed, sr=AUDIO_SAMPLE_RATE, hop_length=HOP_LENGTH, frame_size=N_FFT, mean=mean, std=std, win_length=WIN_LENGTH, window=WINDOW, center=CENTER, pad_mode=PAD_MODE, power=POWER, n_iter=N_ITER)
    
    return reconstructed_signal

# Test the autoencoder reconstruction
for audio in padded_train_ds.take(5):
    audio_normalized, mean, std = normalize_standard(audio.numpy())
    
    print("Original Audio:")
    ipd.display(ipd.Audio(audio_normalized, rate=AUDIO_SAMPLE_RATE))

    # Use autoencoder to compress and decompress the audio
    reconstructed_signal = compress_and_decompress(audio, mean, std)
    
    print("Reconstructed Audio shape:", reconstructed_signal.shape)
    print("Reconstructed Audio:")
    ipd.display(ipd.Audio(reconstructed_signal, rate=AUDIO_SAMPLE_RATE))


Original Audio:


1/1 [==============================] - 0s 14ms/step
Compressed and decompressed shape: (1, 128, 32, 1)
Reconstructed Audio shape: (1, 15872)
Reconstructed Audio:


Original Audio:


1/1 [==============================] - 0s 13ms/step
Compressed and decompressed shape: (1, 128, 32, 1)
Reconstructed Audio shape: (1, 15872)
Reconstructed Audio:


Original Audio:


1/1 [==============================] - 0s 21ms/step
Compressed and decompressed shape: (1, 128, 32, 1)
Reconstructed Audio shape: (1, 15872)
Reconstructed Audio:


Original Audio:


1/1 [==============================] - 0s 22ms/step
Compressed and decompressed shape: (1, 128, 32, 1)
Reconstructed Audio shape: (1, 15872)
Reconstructed Audio:


Original Audio:


1/1 [==============================] - 0s 26ms/step
Compressed and decompressed shape: (1, 128, 32, 1)
Reconstructed Audio shape: (1, 15872)
Reconstructed Audio:


In [111]:
autoencoder.save('models/autoencoder_model.h5')

c:\Users\Dodek\anaconda3\envs\GlobalDsEnv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [112]:
from tensorflow.keras.models import load_model

# Load the entire autoencoder model
loaded_autoencoder = load_model('models/autoencoder_model.h5')

### Current network

In [50]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Encoder
def build_encoder(input_shape):
    encoder_input = layers.Input(shape=input_shape, name='encoder_input')

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoder_input)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    encoder_output = layers.Flatten()(x)
    encoder_output = layers.Dense(128, activation='relu')(encoder_output)

    return models.Model(encoder_input, encoder_output, name='encoder')

# Define the Decoder
def build_decoder(encoded_shape):
    decoder_input = layers.Input(shape=(encoded_shape,), name='decoder_input')

    x = layers.Dense(16 * 16 * 32, activation='relu')(decoder_input)
    x = layers.Reshape((16, 16, 32))(x)
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)

    # Add a Cropping layer to match the exact input shape
    x = layers.Cropping2D(cropping=((0, 0), (0, 2)))(x)

    decoder_output = layers.Conv2DTranspose(1, (3, 3), activation='relu', padding='same')(x)

    return models.Model(decoder_input, decoder_output, name='decoder')

# Define the Autoencoder
def build_autoencoder(input_shape):
    encoder = build_encoder(input_shape)
    decoder = build_decoder(encoder.output_shape[1])

    autoencoder_input = layers.Input(shape=input_shape, name='autoencoder_input')
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)

    return models.Model(autoencoder_input, decoded, name='autoencoder')

# Input shape for the spectrograms
input_shape = (128, 126, 1)  # Example shape, modify as needed

# Build and compile the autoencoder
autoencoder = build_autoencoder(input_shape)
autoencoder.compile(optimizer='adam', loss='mse')

# Summary of the model
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 autoencoder_input (InputLa  [(None, 128, 126, 1)]     0         
 yer)                                                            
                                                                 
 encoder (Functional)        (None, 128)               1142240   
                                                                 
 decoder (Functional)        (None, 128, 126, 1)       1159521   
                                                                 
Total params: 2301761 (8.78 MB)
Trainable params: 2301761 (8.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Previous network

In [52]:
from tensorflow.keras import layers, models
from tensorflow.keras import backend

input_shape = (n_mels, 126, 1)

# Encoder
encoder_input = layers.Input(shape=input_shape)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
shape_before_dense = backend.int_shape(x)[1:]
x = layers.Flatten()(x)
encoded = layers.Dense(20)(x)

encoder = tf.keras.Model(encoder_input, encoded, name="encoder")

# Decoder
decoder_input = layers.Input(shape=20, name="decoder_input")
x = layers.Dense(np.prod(shape_before_dense), name="decoder_dense")(decoder_input)
x = layers.Reshape(target_shape=shape_before_dense)(x)
x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='relu', padding='same')(x)

decoder = tf.keras.Model(decoder_input, decoded, name="decoder")

In [53]:
from source.constants import OPTIMIZER, LOSS

autoencoder = tf.keras.Model(encoder_input, decoder(encoder(encoder_input)), name="autoencoder")
autoencoder.compile(optimizer=OPTIMIZER, loss=LOSS)
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 126, 1)]     0         
                                                                 
 encoder (Functional)        (None, 20)                2582404   
                                                                 
 decoder (Functional)        (None, 128, 126, 1)       2711985   
                                                                 
Total params: 5294389 (20.20 MB)
Trainable params: 5294389 (20.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training

In [54]:
history = autoencoder.fit(ds_train, epochs=20, verbose=1)

Epoch 1/20
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape: (16000,)
Audio shape:

InvalidArgumentError: Graph execution error:

Detected at node 'autoencoder/encoder/dense_4/BiasAdd' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\Users\Dodek\anaconda3\envs\GlobalDsEnv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Dodek\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\Dodek\anaconda3\envs\GlobalDsEnv\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\Users\Dodek\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Dodek\anaconda3\envs\GlobalDsEnv\Lib\asyncio\windows_events.py", line 321, in run_forever
      super().run_forever()
    File "c:\Use

Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Audio shape: (16000,)
Spectrogram shape: (128, 32)
Au